In [2]:
! pip install transformers
! pip install datasets
! pip install torcheval
! pip install pytorch-ignite
! pip install evaluate
! pip install rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 18.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 6.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-non

In [3]:
import transformers as T
from transformers import AutoTokenizer, AutoModel, AutoModelForSeq2SeqLM, GPT2Tokenizer, GPT2LMHeadModel
from datasets import load_dataset, Split, load_metric
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from tqdm import tqdm
# from ignite.metrics import Rouge
import re
device = "cuda" if torch.cuda.is_available() else "cpu"

# Objective: Chinese Text Summarization

## T5 Model

In [ ]:
t5_model = AutoModelForSeq2SeqLM.from_pretrained("ozcangundes/mt5-small-turkish-summarization", cache_dir="./cache/").to(device)
t5_tokenizer = AutoTokenizer.from_pretrained("ozcangundes/mt5-small-turkish-summarization", cache_dir="./cache/")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/388 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


### 資料處理, Tokenize the Text Function

In [ ]:
def get_tensor(sample):
    # 將模型的輸入和ground truth打包成Tensor
    model_inputs = t5_tokenizer.batch_encode_plus([each["text"] for each in sample], padding=True, truncation=True, max_length=512, return_tensors="pt")
    model_outputs = t5_tokenizer.batch_encode_plus([each["summary"] for each in sample], padding=True, truncation=True, max_length=128, return_tensors="pt")
    return model_inputs["input_ids"].to(device), model_outputs["input_ids"].to(device)


class SummarizationDataset(Dataset):
    def __init__(self, split="train"):
        super().__init__()
        assert split in ["train", "validation", "test"]

        #data_limits = {
        #    "train": "train[:50000]",
        #    "validation": "validation[:6000]",
        #    "test": "test[:1000]"
        #}

        # no cache
        df = load_dataset("hugcyp/LCSTS", split=split ).to_pandas()
        #df = load_dataset("hugcyp/LCSTS", split=data_limits[split], cache_dir="./cache/").to_pandas()

        # 由於數據太大，適當切割資料集
        if split == "train":
          df = df.iloc[:15000]
        elif split == "validation":
          df = df.iloc[:1000]
        elif split == "test":
          df = df.iloc[:100]

        #if split == "train":    # for testing
        #  df = df.iloc[:100]
        #elif split == "validation":
        #  df = df.iloc[:10]
        #elif split == "test":
        #  df = df.iloc[:10]

        self.data = df.to_dict('records')

    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return len(self.data)

In [ ]:
dataset = SummarizationDataset(split="train")
data_sample = dataset.data[:5]

for sample in data_sample:
    print(f"Text: {sample['text']}\nSummary: {sample['summary']}\n")

Generating train split:   0%|          | 0/2400591 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8685 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/725 [00:00<?, ? examples/s]

Text: 新华社受权于18日全文播发修改后的《中华人民共和国立法法》，修改后的立法法分为“总则”“法律”“行政法规”“地方性法规、自治条例和单行条例、规章”“适用与备案审查”“附则”等6章，共计105条。
Summary: 修改后的立法法全文公布

Text: 一辆小轿车，一名女司机，竟造成9死24伤。日前，深圳市交警局对事故进行通报：从目前证据看，事故系司机超速行驶且操作不当导致。目前24名伤员已有6名治愈出院，其余正接受治疗，预计事故赔偿费或超一千万元。
Summary: 深圳机场9死24伤续：司机全责赔偿或超千万

Text: 1月18日，习近平总书记对政法工作作出重要指示：2014年，政法战线各项工作特别是改革工作取得新成效。新形势下，希望全国政法机关主动适应新形势，为公正司法和提高执法司法公信力提供有力制度保障。
Summary: 孟建柱：主动适应形势新变化提高政法机关服务大局的能力

Text: 针对央视3·15晚会曝光的电信行业乱象，工信部在公告中表示，将严查央视3·15晚会曝光通信违规违法行为。工信部称，已约谈三大运营商有关负责人，并连夜责成三大运营商和所在省通信管理局进行调查，依法依规严肃处理。
Summary: 工信部约谈三大运营商严查通信违规

Text: 国家食药监管总局近日发布《食品召回管理办法》，明确：食用后已经或可能导致严重健康损害甚至死亡的，属一级召回，食品生产者应在知悉食品安全风险后24小时内启动召回，且自公告发布之日起10个工作日内完成召回。
Summary: 食品一级召回限24小时内启动10工作日完成



### 超參數 + Split the Data + Tokenize the Text

In [ ]:
lr = 1e-4
epochs = 2
optimizer = AdamW(t5_model.parameters(), lr=lr)
train_batch_size = 300
validation_batch_size = 300
summarization_train = DataLoader(SummarizationDataset(split="train"), collate_fn=get_tensor, batch_size=train_batch_size, shuffle=True)
summarization_validation = DataLoader(SummarizationDataset(split="validation"), collate_fn=get_tensor, batch_size=validation_batch_size, shuffle=False)
#rouge = Rouge(variants=["L", 2], multiref="best")

### Evaluate while Training

In [ ]:
def evaluate(model, dataloader, tokenizer, device):
    model.eval()  # evaluate mode
    rouge = load_metric('rouge')
    predictions = []
    references = []
    pbar = tqdm(dataloader, desc="Evaluating")

    with torch.no_grad():
        for batch in pbar:
            inputs = batch[0].to(device)
            labels = batch[1].to(device)

            # 生成summary
            outputs = model.generate(inputs, max_length=128, num_beams=5, early_stopping=True)

            # 將生成的summary和真實summary解碼回去
            decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True, clean_up_tokenization_spaces=True)
            decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True, clean_up_tokenization_spaces=True)

            predictions.extend(decoded_preds)
            references.extend(decoded_labels)

    # 算rouge分數
    result = rouge.compute(predictions=predictions, references=references, use_stemmer=True)

    # 印出rouge分數, key=rouge-1, rouge-2, rouge-L, rouge-Lsum
    print("Evaluation results:")
    for key in result.keys():
        print(f"{key}: Precision: {result[key].mid.precision:.4f}, Recall: {result[key].mid.recall:.4f}, F1: {result[key].mid.fmeasure:.4f}")

    return result

In [ ]:
'''
# just for testing
all_results = []

for ep in range(epochs):
    pbar = tqdm(summarization_train)
    pbar.set_description(f"Training epoch [{ep+1}/{epochs}]")
    for inputs, targets in pbar:
        optimizer.zero_grad()
        loss = t5_model(input_ids=inputs, labels=targets).loss
        loss.backward()
        optimizer.step()
        pbar.set_postfix(loss = loss.item())
    #torch.save(t5_model, f'./saved_models/ep{ep}.mod')
    epoch_results = evaluate(t5_model, summarization_validation, t5_tokenizer, device)
    all_results.append(epoch_results)
    print(f"Metrics Score on Epoch {ep}: {epoch_results}")
'''

### Train/Finetune the Model

In [ ]:
for ep in range(epochs):
    pbar = tqdm(summarization_train)
    pbar.set_description(f"Training epoch [{ep+1}/{epochs}]")
    for inputs, targets in pbar:
        optimizer.zero_grad()
        loss = t5_model(input_ids=inputs, labels=targets).loss
        loss.backward()
        optimizer.step()
        pbar.set_postfix(loss = loss.item())
    torch.save(t5_model, f'./saved_models/ep{ep}.mod')
    print(f"Metrics Score on Epoch {ep}:", evaluate(t5_model, summarization_validation, t5_tokenizer, device))

Training epoch [1/2]: 100%|██████████| 50/50 [10:33<00:00, 12.67s/it, loss=3.34]
<ipython-input-7-8f462c1d96da>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric('rouge')
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Evaluating: 100%|██████████| 4/4 [04:08<00:00, 62.02s/it]


Evaluation results:
rouge1: Precision: 0.0815, Recall: 0.0835, F1: 0.0797
rouge2: Precision: 0.0063, Recall: 0.0070, F1: 0.0062
rougeL: Precision: 0.0810, Recall: 0.0826, F1: 0.0790
rougeLsum: Precision: 0.0816, Recall: 0.0835, F1: 0.0797
Metrics Score on Epoch 0: {'rouge1': AggregateScore(low=Score(precision=0.06541666666666666, recall=0.06606583333333334, fmeasure=0.06359285714285713), mid=Score(precision=0.08148333333333334, recall=0.08353333333333332, fmeasure=0.07974365079365081), high=Score(precision=0.09851041666666667, recall=0.10067083333333332, fmeasure=0.09646996031746034)), 'rouge2': AggregateScore(low=Score(precision=0.0021666666666666666, recall=0.0025, fmeasure=0.0023950000000000043), mid=Score(precision=0.006333333333333333, recall=0.007, fmeasure=0.006233333333333334), high=Score(precision=0.011666666666666665, recall=0.012, fmeasure=0.011135)), 'rougeL': AggregateScore(low=Score(precision=0.06531625, recall=0.06672750000000001, fmeasure=0.06385531746031745), mid=Score

Training epoch [2/2]: 100%|██████████| 50/50 [32:04<00:00, 38.50s/it, loss=1.85]
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
Evaluating: 100%|██████████| 4/4 [03:52<00:00, 58.16s/it]


Evaluation results:
rouge1: Precision: 0.0844, Recall: 0.0860, F1: 0.0822
rouge2: Precision: 0.0103, Recall: 0.0110, F1: 0.0102
rougeL: Precision: 0.0842, Recall: 0.0858, F1: 0.0819
rougeLsum: Precision: 0.0846, Recall: 0.0862, F1: 0.0824
Metrics Score on Epoch 1: {'rouge1': AggregateScore(low=Score(precision=0.06780833333333333, recall=0.069, fmeasure=0.06631964285714285), mid=Score(precision=0.0844, recall=0.086, fmeasure=0.08219761904761905), high=Score(precision=0.10241791666666666, recall=0.10420416666666667, fmeasure=0.09884523809523812)), 'rouge2': AggregateScore(low=Score(precision=0.004583333333333333, recall=0.005, fmeasure=0.0044666666666666665), mid=Score(precision=0.010250000000000002, recall=0.011, fmeasure=0.0102), high=Score(precision=0.01667083333333333, recall=0.017, fmeasure=0.01653333333333333)), 'rougeL': AggregateScore(low=Score(precision=0.06840875000000002, recall=0.0684975, fmeasure=0.0658465476190476), mid=Score(precision=0.08424166666666667, recall=0.08583333

## GPT-2 Model

In [4]:
gpt2_model = GPT2LMHeadModel.from_pretrained("benjamin/gpt2-wechsel-chinese").to(device)
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("benjamin/gpt2-wechsel-chinese")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/947 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/665M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/251 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.45M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.67M [00:00<?, ?B/s]

### Finetune

In [8]:
PAD_TOKEN_DICT = {'pad_token': '<pad>'}
gpt2_tokenizer.add_special_tokens(PAD_TOKEN_DICT)
gpt2_model.resize_token_embeddings(len(gpt2_tokenizer))

Embedding(50258, 768)

### 資料處理, Tokenize the Text Function


In [12]:
def get_tensor(sample):
    # Prepare model inputs and outputs with uniform padding
    texts = [s['text_summary'] for s in sample]
    summaries = [s['summary'] for s in sample]

    # Encode texts with maximum length of 512
    model_inputs = gpt2_tokenizer.batch_encode_plus(texts, padding='max_length', truncation=True, max_length=512, return_tensors="pt", add_special_tokens=True)
    # Encode summaries with maximum length of 512 to match the inputs and ensure uniform padding
    model_outputs = gpt2_tokenizer.batch_encode_plus(summaries, padding='max_length', truncation=True, max_length=512, return_tensors="pt", add_special_tokens=True)

    # Make sure that labels are padded to the same length as inputs for consistency
    labels = model_outputs["input_ids"].to(device)
    if labels.shape[1] < model_inputs["input_ids"].shape[1]:
        labels = torch.nn.functional.pad(
            labels, (0, model_inputs["input_ids"].shape[1] - labels.shape[1]), value=gpt2_tokenizer.pad_token_id
        )

    return model_inputs["input_ids"].to(device), model_inputs['attention_mask'].to(device), labels


    # 將模型的輸入和ground truth打包成Tensor
    #model_inputs = gpt2_tokenizer.batch_encode_plus([each["text_summary"] for each in sample], padding=True, truncation=True, max_length=512, return_tensors="pt", add_special_tokens=True)
    #model_outputs = gpt2_tokenizer.batch_encode_plus([each["summary"] for each in sample], padding=True, truncation=True, max_length=128, return_tensors="pt", add_special_tokens=True)
    #return model_inputs["input_ids"].to(device), model_inputs['attention_mask'].to(device), model_outputs["input_ids"].to(device)


class SummarizationDataset(Dataset):
    def __init__(self, split="train"):
        super().__init__()
        assert split in ["train", "validation", "test"]

        #data_limits = {
        #    "train": "train[:50000]",
        #    "validation": "validation[:6000]",
        #    "test": "test[:1000]"
        #}

        df = load_dataset("hugcyp/LCSTS", split=split ).to_pandas()
        #df = load_dataset("hugcyp/LCSTS", split=data_limits[split], cache_dir="./cache/").to_pandas()

        # 由於數據太大，適當切割資料集
        if split == "train":
          df = df.iloc[:15000]
        elif split == "validation":
          df = df.iloc[:1000]
        elif split == "test":
          df = df.iloc[:100]

        #if split == "train":    # for testing
        #  df = df.iloc[:100]
        #elif split == "validation":
        #  df = df.iloc[:10]
        #elif split == "test":
        #  df = df.iloc[:10]

        # 拼接text和summary，用<eos>區隔+結尾
        df['text_summary'] = df['text'] + gpt2_tokenizer.eos_token + df['summary'] + gpt2_tokenizer.eos_token

        self.data = df.to_dict('records')

    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return len(self.data)

In [32]:
dataset = SummarizationDataset(split="train")
data_sample = dataset.data[:5]

for sample in data_sample:
    print(f"Text: {sample['text']}\nSummary: {sample['summary']}\n")

Text: 新华社受权于18日全文播发修改后的《中华人民共和国立法法》，修改后的立法法分为“总则”“法律”“行政法规”“地方性法规、自治条例和单行条例、规章”“适用与备案审查”“附则”等6章，共计105条。
Summary: 修改后的立法法全文公布

Text: 一辆小轿车，一名女司机，竟造成9死24伤。日前，深圳市交警局对事故进行通报：从目前证据看，事故系司机超速行驶且操作不当导致。目前24名伤员已有6名治愈出院，其余正接受治疗，预计事故赔偿费或超一千万元。
Summary: 深圳机场9死24伤续：司机全责赔偿或超千万

Text: 1月18日，习近平总书记对政法工作作出重要指示：2014年，政法战线各项工作特别是改革工作取得新成效。新形势下，希望全国政法机关主动适应新形势，为公正司法和提高执法司法公信力提供有力制度保障。
Summary: 孟建柱：主动适应形势新变化提高政法机关服务大局的能力

Text: 针对央视3·15晚会曝光的电信行业乱象，工信部在公告中表示，将严查央视3·15晚会曝光通信违规违法行为。工信部称，已约谈三大运营商有关负责人，并连夜责成三大运营商和所在省通信管理局进行调查，依法依规严肃处理。
Summary: 工信部约谈三大运营商严查通信违规

Text: 国家食药监管总局近日发布《食品召回管理办法》，明确：食用后已经或可能导致严重健康损害甚至死亡的，属一级召回，食品生产者应在知悉食品安全风险后24小时内启动召回，且自公告发布之日起10个工作日内完成召回。
Summary: 食品一级召回限24小时内启动10工作日完成



### 超參數 + Split the Data + Tokenize the Text

In [33]:
lr = 1e-3
epochs = 2
optimizer = AdamW(gpt2_model.parameters(), lr=lr)
train_batch_size = 200
validation_batch_size = 200
summarization_train = DataLoader(SummarizationDataset(split="train"), collate_fn=get_tensor, batch_size=train_batch_size, shuffle=True)
summarization_validation = DataLoader(SummarizationDataset(split="validation"), collate_fn=get_tensor, batch_size=validation_batch_size, shuffle=False)
#rouge = Rouge(variants=["L", 2], multiref="best")

### Evaluate while Training

In [34]:
def evaluate(model, dataloader, tokenizer, device):
    model.eval()  # evaluate mode
    rouge = load_metric('rouge')
    predictions = []
    references = []
    pbar = tqdm(dataloader, desc="Evaluating")

    with torch.no_grad():
        for batch in pbar:
            inputs = batch[0].to(device)
            masks = batch[1].to(device)
            labels = batch[2].to(device)
            outputs = model.generate(inputs, attention_mask=masks, max_new_tokens=50, num_beams=5, early_stopping=True)
            decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True, clean_up_tokenization_spaces=True)
            decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True, clean_up_tokenization_spaces=True)

            predictions.extend(decoded_preds)
            references.extend(decoded_labels)

    # 算rouge分數
    result = rouge.compute(predictions=predictions, references=references, use_stemmer=True)

    # 印出rouge分數, key=rouge-1, rouge-2, rouge-L, rouge-Lsum
    print("Evaluation results:")
    for key in result.keys():
        print(f"{key}: Precision: {result[key].mid.precision:.4f}, Recall: {result[key].mid.recall:.4f}, F1: {result[key].mid.fmeasure:.4f}")

    return result

### Train the Model




In [ ]:
for ep in range(epochs):
    pbar = tqdm(summarization_train)
    pbar.set_description(f"Training epoch [{ep+1}/{epochs}]")
    for batch in pbar:
        inputs = batch[0].to(device)
        masks = batch[1].to(device)
        labels = batch[2].to(device)

        # Print shapes of tensors(for debug)
        #print("Input IDs shape:", inputs.shape)
        #print("Masks shape:", masks.shape)
        #print("Labels shape:", labels.shape)

        optimizer.zero_grad()
        loss = gpt2_model(input_ids=inputs, attention_mask=masks, labels=labels).loss
        loss.backward()
        optimizer.step()
        pbar.set_postfix(loss = loss.item())

    #torch.save(gpt2_model, f'./saved_models/ep{ep}.mod')
    print(f"Metrics Score on Epoch {ep}:", evaluate(gpt2_model, summarization_validation, gpt2_tokenizer, device))